In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import codecs
import csv
import re
import sys

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [2]:
#nltk.download()

In [3]:
BASE_DIR = 'Quora-Question-Pairs/'
EMBEDDING_FILE = BASE_DIR + 'GoogleNews-vectors-negative300.bin'
TRAIN_FILE = BASE_DIR + 'train.csv'
TEST_FILE = BASE_DIR + 'test.csv'

In [4]:
MAX_SEQUENCE_LENGTH = 30
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1

num_lstm = np.random.randint(175, 275)
num_dense = np.random.randint(100, 150)
rate_drop_lstm = 0.15 + np.random.rand() * 0.25
rate_drop_dense = 0.15 + np.random.rand() * 0.25

In [5]:
act = 'relu'
re_weight = True

In [6]:
STAMP = 'lstm_%d_%d_%.2f_%.2f'%(num_lstm, num_dense, rate_drop_lstm, \
        rate_drop_dense)

In [7]:
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [8]:
print(word2vec.vocab['School'])

Vocab(count:2999004, index:996)


In [9]:
word2vec.vocab

{'</s>': <gensim.models.keyedvectors.Vocab at 0x7ffa183f16d8>,
 'in': <gensim.models.keyedvectors.Vocab at 0x7ffa2445b7f0>,
 'for': <gensim.models.keyedvectors.Vocab at 0x7ffa2445ba20>,
 'that': <gensim.models.keyedvectors.Vocab at 0x7ffa2445ba90>,
 'is': <gensim.models.keyedvectors.Vocab at 0x7ffa183d76a0>,
 'on': <gensim.models.keyedvectors.Vocab at 0x7ffa1815bfd0>,
 '##': <gensim.models.keyedvectors.Vocab at 0x7ffa1818a860>,
 'The': <gensim.models.keyedvectors.Vocab at 0x7ffa1818aa90>,
 'with': <gensim.models.keyedvectors.Vocab at 0x7ffa1818a908>,
 'said': <gensim.models.keyedvectors.Vocab at 0x7ffa1818aa58>,
 'was': <gensim.models.keyedvectors.Vocab at 0x7ffa1818aac8>,
 'the': <gensim.models.keyedvectors.Vocab at 0x7ffa1818aba8>,
 'at': <gensim.models.keyedvectors.Vocab at 0x7ffa1818abe0>,
 'not': <gensim.models.keyedvectors.Vocab at 0x7ffa1818ac50>,
 'as': <gensim.models.keyedvectors.Vocab at 0x7ffa1818ab70>,
 'it': <gensim.models.keyedvectors.Vocab at 0x7ffa1818ad68>,
 'be': <gen

In [10]:
def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

In [11]:
texts_1 = [] 
texts_2 = []
labels = []
with codecs.open(TRAIN_FILE, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    header = next(reader)
    for values in reader:
        texts_1.append(text_to_wordlist(values[3]))
        texts_2.append(text_to_wordlist(values[4]))
        labels.append(int(values[5]))
print('Found %s texts in train.csv' % len(texts_1))

Found 404290 texts in train.csv


In [12]:
test_texts_1 = []
test_texts_2 = []
test_ids = []
with codecs.open(TEST_FILE, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    header = next(reader)
    for values in reader:
        test_texts_1.append(text_to_wordlist(values[1]))
        test_texts_2.append(text_to_wordlist(values[2]))
        test_ids.append(values[0])
print('Found %s texts in test.csv' % len(test_texts_1))

Found 2345796 texts in test.csv


In [14]:
tokenizer.fit_on_texts(texts_1 + texts_2 + test_texts_1 + test_texts_2)

In [15]:
sequences_1 = tokenizer.texts_to_sequences(texts_1)
sequences_2 = tokenizer.texts_to_sequences(texts_2)
test_sequences_1 = tokenizer.texts_to_sequences(test_texts_1)
test_sequences_2 = tokenizer.texts_to_sequences(test_texts_2)

In [99]:
test_sequences_2

[[15, 57, 729, 419, 913, 17739, 12, 31, 913, 4254, 248, 2042, 826, 171],
 [5, 69, 267, 21, 239, 3799, 1857],
 [2, 16, 610, 73, 8, 255],
 [2, 1674, 11114],
 [5, 76, 13, 4, 95, 672],
 [4, 6121, 4, 47, 2075, 10, 964, 1168, 34, 124],
 [5, 13, 4, 414, 18, 1426, 603, 329, 7, 48, 324],
 [5,
  38,
  1,
  1678,
  617,
  8,
  3557,
  249,
  12,
  259,
  3875,
  24492,
  1096,
  38,
  3439,
  232,
  73],
 [2, 11, 33, 10, 1, 963, 333, 136, 10, 83, 59],
 [13, 6, 1127, 132, 273, 452, 6, 536, 103, 132, 123],
 [2, 11, 1, 19, 156, 14, 150],
 [2,
  308,
  767,
  39,
  133,
  126,
  290,
  4722,
  818,
  243,
  172,
  3370,
  11408,
  491,
  233,
  2413,
  875,
  20,
  43,
  12,
  23065],
 [29,
  10078,
  743,
  3035,
  12,
  345,
  2561,
  3035,
  24,
  131,
  3035,
  11,
  4365,
  2,
  3,
  1,
  1609,
  10,
  216,
  94,
  743,
  3035,
  12,
  121,
  2561,
  3035],
 [5, 9, 24160, 21839, 81],
 [7, 1801, 1543, 2554, 15, 414, 13844, 15553, 6275, 2554, 374, 1801],
 [29, 4, 625, 22, 122, 65020, 508],
 [2, 3,

In [3]:
len(sequences_1)

NameError: name 'sequences_1' is not defined

In [4]:
len(texts_1)

NameError: name 'texts_1' is not defined

In [18]:
sequences_1[0]

[2, 3, 1, 1254, 61, 1254, 2921, 8, 578, 7, 759, 370, 7, 35]

In [19]:
texts_1[0]

'what is the step by step guide to invest in share market in india '

In [20]:
word_index = tokenizer.word_index

In [21]:
data_1 = pad_sequences(sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
data_2 = pad_sequences(sequences_2, maxlen=MAX_SEQUENCE_LENGTH)

In [22]:
test_ids = np.array(test_ids)

In [23]:
nb_words = min(MAX_NB_WORDS, len(word_index))+1

In [24]:
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))

In [25]:
embedding_matrix.shape

(120500, 300)

In [89]:
word2vec

In [90]:
word2vec.word_vec('how')

array([ 0.26953125,  0.0859375 ,  0.09423828,  0.04101562, -0.18359375,
        0.12890625,  0.0625    , -0.10058594,  0.1484375 , -0.06591797,
        0.13671875, -0.19042969, -0.12255859, -0.05908203, -0.29492188,
        0.08496094, -0.02258301,  0.20410156, -0.05151367, -0.03295898,
       -0.08056641,  0.02624512,  0.02758789,  0.10107422, -0.01708984,
        0.10693359,  0.07324219, -0.11425781,  0.02111816,  0.06176758,
       -0.11035156,  0.29296875, -0.08789062, -0.19140625,  0.05834961,
        0.12695312,  0.22949219,  0.02575684,  0.03344727,  0.0255127 ,
        0.02624512,  0.0279541 ,  0.04711914, -0.0456543 , -0.06835938,
        0.05981445, -0.00946045,  0.00500488, -0.0144043 ,  0.08300781,
        0.0625    ,  0.03955078,  0.03515625, -0.17675781,  0.08300781,
       -0.06738281, -0.14550781,  0.07373047,  0.18847656,  0.0559082 ,
        0.09130859,  0.06933594, -0.15136719, -0.02307129, -0.07275391,
       -0.18457031,  0.06201172,  0.125     , -0.01055908, -0.02

In [28]:
for word, i in word_index.items():
    if word in word2vec.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)

In [29]:
embedding_matrix.shape

(120500, 300)

In [30]:
np.sum(np.sum(embedding_matrix, axis=1) == 0)

61789

In [31]:
perm = np.random.permutation(len(data_1))
idx_train = perm[:int(len(data_1)*(1-VALIDATION_SPLIT))]
idx_val = perm[int(len(data_1)*(1-VALIDATION_SPLIT)):]

In [32]:
idx_train.shape

(363861,)

In [33]:
idx_val.shape

(40429,)

In [34]:
idx_val

array([374805, 247577,  43158, ...,  37098, 319677, 341329])

In [45]:
labels = np.asarray(labels)

In [46]:
data_1_train = np.vstack((data_1[idx_train], data_2[idx_train]))
data_2_train = np.vstack((data_2[idx_train], data_1[idx_train]))
labels_train = np.concatenate((labels[idx_train], labels[idx_train]))

In [47]:
data_1_train.shape

(727722, 30)

In [48]:
data_2_train.shape

(727722, 30)

In [55]:
labels_train.shape

(727722,)

In [57]:
data_1_val = np.vstack((data_1[idx_val], data_2[idx_val]))
data_2_val = np.vstack((data_2[idx_val], data_1[idx_val]))
labels_val = np.concatenate((labels[idx_val], labels[idx_val]))

In [58]:
weight_val = np.ones(len(labels_val))

In [60]:
weight_val.shape

(80858,)

In [62]:
if re_weight:
    weight_val *= 0.472001959
    weight_val[labels_val==0] = 1.309028344

In [63]:
weight_val

array([ 0.47200196,  1.30902834,  1.30902834, ...,  0.47200196,
        0.47200196,  0.47200196])

## Create model

In [66]:
embedding_layer = Embedding(nb_words,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False)

In [67]:
lstm_layer = LSTM(num_lstm, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)

In [76]:
sequence_1_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1 = lstm_layer(embedded_sequences_1)
print(x1.shape)
sequence_2_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences_2 = embedding_layer(sequence_2_input)
y1 = lstm_layer(embedded_sequences_2)

(?, 234)


In [69]:
merged = concatenate([x1, y1])
merged = Dropout(rate_drop_dense)(merged)
merged = BatchNormalization()(merged)

merged = Dense(num_dense, activation=act)(merged)
merged = Dropout(rate_drop_dense)(merged)
merged = BatchNormalization()(merged)

In [70]:
preds = Dense(1, activation='sigmoid')(merged)

In [71]:
if re_weight:
    class_weight = {0: 1.309028344, 1: 0.472001959}
else:
    class_weight = None

## Train

In [73]:
model = Model(inputs=[sequence_1_input, sequence_2_input], \
        outputs=preds)
model.compile(loss='binary_crossentropy',
        optimizer='nadam',
        metrics=['acc'])

In [75]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 30)            0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 30)            0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 30, 300)       36150000                                     
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 234)           500760                                       
___________________________________________________________________________________________

In [74]:
print(STAMP)

lstm_234_119_0.32_0.33


In [77]:
early_stopping =EarlyStopping(monitor='val_loss', patience=3)
bst_model_path = STAMP + '.h5'

In [79]:
model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)

In [80]:
model.fit([data_1_train, data_2_train], labels_train, \
        validation_data=([data_1_val, data_2_val], labels_val, weight_val), \
        epochs=200, batch_size=2048, shuffle=True, \
        class_weight=class_weight, callbacks=[early_stopping, model_checkpoint])

Train on 727722 samples, validate on 80858 samples
Epoch 1/200
727722/727722 [==============================] - 107s - loss: 0.4273 - acc: 0.6741 - val_loss: 0.3854 - val_acc: 0.6508
Epoch 2/200
727722/727722 [==============================] - 103s - loss: 0.3646 - acc: 0.7039 - val_loss: 0.3393 - val_acc: 0.7090
Epoch 3/200
727722/727722 [==============================] - 103s - loss: 0.3431 - acc: 0.7222 - val_loss: 0.3209 - val_acc: 0.7338
Epoch 4/200
727722/727722 [==============================] - 103s - loss: 0.3284 - acc: 0.7356 - val_loss: 0.3132 - val_acc: 0.7515
Epoch 5/200
727722/727722 [==============================] - 103s - loss: 0.3168 - acc: 0.7466 - val_loss: 0.3104 - val_acc: 0.7728
Epoch 6/200
727722/727722 [==============================] - 103s - loss: 0.3086 - acc: 0.7546 - val_loss: 0.2999 - val_acc: 0.7787
Epoch 7/200
727722/727722 [==============================] - 104s - loss: 0.3008 - acc: 0.7627 - val_loss: 0.2894 - val_acc: 0.7705
Epoch 8/200
727722/727722

KeyboardInterrupt: 